In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import time
import yaml
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

import sys
import gc
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '/workspace/PaddleOCR/')))
import tools.infer.utility as utility
from ppocr.utils.logging import get_logger
from tools.infer.utility import init_args, parse_args
from paddleocr import PaddleOCR, draw_ocr

import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '/workspace/csc_competition/')))
from det_pred import TextDetector, mult2point, four_point_transform
from rec_pred import Paddlepreditor,yi_merge

/opt/conda/envs/paddle/lib/python3.7/site-packages/skimage/morphology/_skeletonize.py:241: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  0, 1, 1, 0, 0, 1, 0, 0, 0], dtype=np.bool)
/opt/conda/envs/paddle/lib/python3.7/site-packages/skimage/morphology/_skeletonize.py:256: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  0, 0, 0, 0, 0, 0, 0, 0, 0], dtype

In [3]:
det_inference_path = '/workspace/PaddleOCR/inference/det_db_1220_th01/Student2/'
rec_inference_path = '/workspace/PaddleOCR/inference/rec_1223/'

In [4]:
def order_points(pts):
	# initialize a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype="float32")
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis=1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis=1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

## load model

In [5]:
from paddleocr import PaddleOCR, draw_ocr
# 模型路径下必须含有model和params文件
ocr = PaddleOCR(lang="chinese_cht",
                det_model_dir=det_inference_path,
                det_db_thresh=0.008,
                #cls_model_dir='/workspace/PaddleOCR/inference/csc_cls_mv3', 
                rec_model_dir=rec_inference_path,  
                rec_char_dict_path="/workspace/PaddleOCR/ppocr/utils/dict/cht_dict.txt",
                #vis_font_path="doc/fonts/chinese_cht.ttf"
                use_angle_cls=True,
                version='PP-OCR', 
                show_log=False
               )

Namespace(benchmark=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/root/.paddleocr/2.2.0.1/ocr/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.008, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/workspace/PaddleOCR/inference/det_db_1220_th01/Student2/', det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_polygon=True, e2e_pgnet_score_thresh=0.5, e2e_pgnet_valid_set='totaltext', enable_mkldnn=False, gpu_mem=500, help='==SUPPRESS==', image_dir=None, ir_optim=True, label_list=['0', '180'], lang='chinese_cht', layout_path_model=

W1228 14:28:45.128319 1540782 analysis_predictor.cc:715] The one-time configuration of analysis predictor failed, which may be due to native predictor called first and its configurations taken effect.
--- Running analysis [ir_graph_build_pass]
--- Running analysis [ir_graph_clean_pass]
--- Running analysis [ir_analysis_pass]
--- Running IR pass [is_test_pass]
--- Running IR pass [simplify_with_basic_ops_pass]
--- Running IR pass [conv_affine_channel_fuse_pass]
--- Running IR pass [conv_eltwiseadd_affine_channel_fuse_pass]
--- Running IR pass [conv_bn_fuse_pass]
I1228 14:28:45.291926 1540782 graph_pattern_detector.cc:91] ---  detected 33 subgraphs
--- Running IR pass [conv_eltwiseadd_bn_fuse_pass]
--- Running IR pass [embedding_eltwise_layernorm_fuse_pass]
--- Running IR pass [multihead_matmul_fuse_pass_v2]
--- Running IR pass [squeeze2_matmul_fuse_pass]
--- Running IR pass [reshape2_matmul_fuse_pass]
--- Running IR pass [flatten2_matmul_fuse_pass]
--- Running IR pass [map_matmul_to_mul

In [6]:
img_path='/workspace/text_spot_rec/data/public/img*'

In [835]:
result=[]
for file in tqdm(glob(img_path)):
    x=ocr.ocr(file, det=True, rec=True, cls=True)
    for i in x:
        result.append([file.split('/')[-1].split('.')[0],
                       i[0][0][0],i[0][0][1],i[0][1][0],i[0][1][1],
                       i[0][2][0],i[0][2][1],i[0][3][0],i[0][3][1],
                       i[1][0],i[1][1]])

100% 5500/5500 [21:44<00:00,  4.22it/s]


In [843]:
out=pd.DataFrame(result)

In [844]:
out=out.sort_values(by=[0,1]).reset_index()

In [845]:
out=out.drop(columns=['index'])

In [847]:
out[1]=out[1].astype(int)
out[2]=out[2].astype(int)
out[3]=out[3].astype(int)
out[4]=out[4].astype(int)
out[5]=out[5].astype(int)
out[6]=out[6].astype(int)
out[7]=out[7].astype(int)
out[8]=out[8].astype(int)

In [850]:
out[11]=out[9].apply(lambda x: x.replace('@','').replace('-','').replace(':','').replace('.','').replace('+',''))

In [859]:
out[[0,1,2,3,4,5,6,7,8,11]].to_csv('output_csv/e2e_rec_result.csv',index=False,header=None)